# Лабораторная работа №1
## Задание:
1. Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).
2. Провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Подключение библиотек


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

### Загрузка данных в датасет


In [2]:
Dataset = pd.read_csv('property_assessment_filtered.csv', low_memory=False)
dataset_rows = Dataset.shape[0]
print(Dataset.head())
print(dataset_rows, '- строк')

   Unnamed: 0        PID        CM_ID     GIS_ID  ST_NUM       ST_NAME  \
0          18  100018002  100018000.0  100018000   239.0  Lexington ST   
1          19  100018004  100018000.0  100018000   239.0  Lexington ST   
2          20  100018006  100018000.0  100018000   239.0  Lexington ST   
3          21  100018008  100018000.0  100018000   239.0  Lexington ST   
4          23  100019002  100019000.0  100019000   241.0  Lexington ST   

          CITY  ZIP_CODE  BLDG_SEQ  NUM_BLDGS  ...  FULL_BTH HLF_BTH KITCHENS  \
0  EAST BOSTON    2128.0         1          1  ...       2.0     0.0      1.0   
1  EAST BOSTON    2128.0         1          1  ...       2.0     0.0      1.0   
2  EAST BOSTON    2128.0         1          1  ...       2.0     0.0      1.0   
3  EAST BOSTON    2128.0         1          1  ...       2.0     0.0      1.0   
4  EAST BOSTON    2128.0         1          1  ...       3.0     0.0      1.0   

  TT_RMS    KITCHEN_TYPE           HEAT_TYPE         AC_TYPE FIREPLA

### Создаём искомый столбец

In [3]:
label_encoder = LabelEncoder()
Dataset['СOND_ENC'] = label_encoder.fit_transform(Dataset['OVERALL_COND'])
print(Dataset[['OVERALL_COND', 'СOND_ENC']].head(10))
print(Dataset['СOND_ENC'].unique())
print(Dataset['OVERALL_COND'].unique())

  OVERALL_COND  СOND_ENC
0  A - Average         0
1  A - Average         0
2  A - Average         0
3  A - Average         0
4     G - Good         4
5     G - Good         4
6     G - Good         4
7  A - Average         0
8  A - Average         0
9  A - Average         0
[0 4 3 1 5 6 2]
['A - Average' 'G - Good' 'F - Fair' 'E - Excellent' 'P - Poor'
 'VG - Very Good' 'EX - Excellent']


### Нормализация данных

In [4]:
Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']] = Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']].applymap(lambda x: x.replace(',', '') if isinstance(x, str) else x).apply(pd.to_numeric, errors='coerce')
numeric_columns = Dataset.select_dtypes(include='number').columns.tolist()
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)
columns_to_fill = ['GROSS_AREA', 'LIVING_AREA', 'TOTAL_VALUE', 'YR_REMODEL']
for column in columns_to_fill:
    mean_value = Dataset[column].mean()
    Dataset[column].fillna(mean_value, inplace=True)
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64508 entries, 0 to 64507
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   64508 non-null  int64  
 1   PID          64508 non-null  int64  
 2   CM_ID        64508 non-null  float64
 3   GIS_ID       64508 non-null  int64  
 4   ST_NUM       64508 non-null  float64
 5   ZIP_CODE     64508 non-null  float64
 6   BLDG_SEQ     64508 non-null  int64  
 7   NUM_BLDGS    64508 non-null  int64  
 8   LUC          64508 non-null  int64  
 9   RES_FLOOR    64508 non-null  float64
 10  GROSS_AREA   64508 non-null  float64
 11  LIVING_AREA  64508 non-null  float64
 12  LAND_VALUE   64508 non-null  int64  
 13  BLDG_VALUE   64508 non-null  int64  
 14  SFYI_VALUE   64508 non-null  int64  
 15  TOTAL_VALUE  64508 non-null  int64  
 16  YR_BUILT     64508 non-null  float64
 17  YR_REMODEL   64508 non-null  float64
 18  BED_RMS      64508 non-null  float64
 19  FULL

### Поиск лучшей метрики и количества соседей

In [ ]:
y = Dataset['СOND_ENC']
X = Dataset[['GROSS_AREA', 'LIVING_AREA', 'TOTAL_VALUE', 'YR_REMODEL']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

knn = KNeighborsClassifier()

param_grid = {'n_neighbors': range(3, 40),
              'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}

grid_search = GridSearchCV(knn, param_grid, cv=2, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_metric = best_params['metric']
best_k = best_params['n_neighbors']

accuracies = grid_search.cv_results_['mean_test_score']
accuracies = np.array(accuracies).reshape(len(param_grid['n_neighbors']), len(param_grid['metric']))

print(f"Лучшие параметры: Количество соседей = {best_k}, Лучшая метрика = {best_metric}")


#### Повторное обучение на лучших метриках и количестве соседей

In [23]:
knn = KNeighborsClassifier(n_neighbors=best_k, metric=best_metric)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели с {best_k} соседями и метрикой {best_metric} : {accuracy}")
report = classification_report(y_test, y_pred, zero_division = 1)
print(report)

Точность модели с 39 соседями и метрикой chebyshev : 0.7320570454193148
              precision    recall  f1-score   support

           0       0.73      1.00      0.85     18926
           1       0.27      0.01      0.02       382
           2       1.00      0.00      0.00       337
           3       1.00      0.00      0.00         8
           4       0.36      0.01      0.02      5913
           5       1.00      0.00      0.00         2
           6       0.00      0.00      1.00       236

    accuracy                           0.73     25804
   macro avg       0.62      0.14      0.27     25804
weighted avg       0.64      0.73      0.63     25804

